In [ ]:
using LinearAlgebra, Plots, LaTeXStrings, Symbolics 

In [ ]:
const E = 2.1E06
const ν = 0.3

global C = ((1 - ν) / ((1 + ν) * (1 - ν))) * E * [1 (ν/(1-ν)) 0; (ν/(1-ν)) 1 0; 0 0 ((1-2*ν)/(2*(1-ν)))];

In [ ]:
const numNodosx = 3
const numNodosy = 3
const numNodosG = numNodosx * numNodosy

global GDLG = numNodosG * 2  # En el caso de elasticidad bidimensional se tiene 2 grados de libertad por nodo

const longx = 3.0
const longy = 3.0

const numElementos = (numNodosx - 1) * (numNodosy - 1);

In [ ]:
begin
    @variables r s
    Dr = Differential(r)
    Ds = Differential(s)

    nFI = 4
    
    h₁ = 1 / 4 * (1 + r) * (1 + s)
    h₂ = 1 / 4 * (1 - r) * (1 + s)
    h₃ = 1 / 4 * (1 - r) * (1 - s)
    h₄ = 1 / 4 * (1 + r) * (1 - s)

    global H = [h₁ h₂ h₃ h₄]
    global ∇H = [transpose([expand_derivatives(Dr(H[i])) for i in 1:nFI]); transpose([expand_derivatives(Ds(H[i])) for i in 1:nFI])]
end

In [ ]:
3333
nNodosInt = 9
GL9Q = [-sqrt(0.6) -sqrt(0.6); 0 -sqrt(0.6); sqrt(0.6) -sqrt(0.6); sqrt(0.6) 0; sqrt(0.6) sqrt(0.6); 
    0 sqrt(0.6); -sqrt(0.6) sqrt(0.6); -sqrt(0.6) 0; 0 0]
wGL9Q = [5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 8/9 8/9];

In [ ]:
# Matriz de conectividad global para elementos bidimensionales de cuatro nodos 
#(El numero de elementos puede variar)
function matrizConectividadGlobal(numNodosx, numNodosy, numElementos)

    matrizConectividad = zeros(Int, numElementos, 4)
    elemento = 1
    
    for j in 1:numNodosy-1
        for i in 1:numNodosx-1
            nodoInferiorIzquierdo::Int = i + ((j - 1) * numNodosx)
            matrizConectividad[elemento, :] = [(nodoInferiorIzquierdo + numNodosx + 1) (nodoInferiorIzquierdo + numNodosx) nodoInferiorIzquierdo (nodoInferiorIzquierdo + 1)]
            elemento += 1
        end
    end

    return matrizConectividad
end 

In [ ]:
function mallador(numNodosx, numNodosy, longx, longy, numElementos)

    numSegmentosx = numNodosx - 1
    numSegmentosy = numNodosy - 1
    dNodosx = longx / numSegmentosx
    dNodosy = longy / numSegmentosy
    numNodosG = numNodosx * numNodosy
    matrizCoordG = zeros(numNodosG, 2)
    Coordy = 0
    nodo = 1

    for j in 1:numNodosy
        Coordx = 0
        for i in 1:numNodosx
            matrizCoordG[nodo, :] = [Coordx Coordy]
            Coordx += dNodosx
            nodo += 1
        end
        Coordy += dNodosy
    end

    return matrizCoordG
end

In [ ]:
function matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)

    numElementos = (numNodosx - 1) * (numNodosy - 1)
    numNodosGlobales = numNodosx * numNodosy
    mGdGL = zeros(Integer, 8, numElementos)
    
    for i in 1:numElementos
        j = 1
        while j <= 4
            zz = mConectividadGlobal[i, :]
            mGdGL[2*j-1, i] = zz[j] * 2 - 1
            mGdGL[2*j, i] = zz[j] * 2
            j += 1
        end
    end
    
    return mGdGL
end

In [ ]:
begin
mConectG = matrizConectividadGlobal(numNodosx, numNodosy, numElementos);
#display(mConectG)
mGdlG = matrizGdGL(numNodosx, numNodosy, mConectG);
#display(mGdlG)
mCoordNodales = mallador(numNodosx, numNodosy, longx, longy, numElementos);
#display(mCoordNodales)
end

In [ ]:
# Matriz de Rigidez Global Lineal *******
function matrizKGL(mConectG, mCoordNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q)

    KGL = zeros(Float64, GDLG, GDLG) 
    mNodalCoords = zeros(4, 2)

    for el in 1:numElementos
        KEL = zeros(Float64, 8, 8)
        GLE = mGdlG[:, el]
        numGLE = length(GLE)

        for nodo in 1:4
         mNodalCoords[nodo, :] = mCoordNodales[mConectG[el, :][nodo], :]
        end

        x = expand((H * mNodalCoords[:, 1])[1])
        y = expand((H * mNodalCoords[:, 2])[1])

        dxdr = expand_derivatives(Dr(x))
        dxds = expand_derivatives(Ds(x))
        dydr = expand_derivatives(Dr(y))
        dyds = expand_derivatives(Ds(y))

        J = [dxdr dydr; dxds dyds] # Jacobiano
        J_det = det(J) # Determinante del Jacobiano
        J_inv = inv(J) # Inversa del Jacobiano
    
        B = transpose(∇H) * J_inv
        
        BL₀ = [B[1, 1] 0 B[2, 1] 0 B[3, 1] 0 B[4, 1] 0
            0 B[1, 2] 0 B[2, 2] 0 B[3, 2] 0 B[4, 2]
            B[1, 2] B[1, 1] B[2, 2] B[2, 1] B[3, 2] B[3, 1] B[4, 2] B[4, 1]]

        for i in 1:9 # Integración en los puntos de Gauss
            Jg_det = substitute(J_det, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Bg = substitute(BL₀, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            KEL += (transpose(Bg) * C * Bg * Jg_det * wGL9Q[i, 1] * wGL9Q[i, 2])
        end

        for i in 1:numGLE
            for j in 1:numGLE
                KGL[GLE[i], GLE[j]] += KEL[i, j].val
            end
        end
    end
    return KGL
end

In [ ]:
KGL = matrizKGL(mConectG, mCoordNodales, C, numElementos, GDLG, mGdlG, GL9Q, wGL9Q)

In [ ]:
#RE = zeros(Number, GDLGlobales, numElementos)
RG = zeros(GDLG)
RG[18] = 3000
RG[14] = 3000
#RG[8] = 3000
#RG[6] = 3000
#RG = [0., 3000., 0., 3000., 0., 0., 0., 0.]
#display(RG)

In [ ]:
# Caso 1 (1 elemento)

# Imponemos condiciones de contorno de Dirichlet
#=
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

display(KG)

U₀ = KGL \ RG # U₀ -> Desplazamientos en ⁰U 
=#

In [ ]:
# Caso 2 (4 Elementos)
# Imponemos condiciones de contorno de Dirichlet

#
# U1x=0 => U1=0
KGL[1, :] .= 0
KGL[:, 1] .= 0
KGL[1, 1] = 1

# U1y=0 => U2=0
KGL[2, :] .= 0
KGL[:, 2] .= 0
KGL[2, 2] = 1

# U2x=0 => U3=0
KGL[3, :] .= 0
KGL[:, 3] .= 0
KGL[3, 3] = 1

# U2y=0 => U4=0
KGL[4, :] .= 0
KGL[:, 4] .= 0
KGL[4, 4] = 1

# U3x=0 => U5=0
KGL[5, :] .= 0
KGL[:, 5] .= 0
KGL[5, 5] = 1

# U3y=0 => U6=0
KGL[6, :] .= 0
KGL[:, 6] .= 0
KGL[6, 6] = 1


U = KGL \ RG;
#

In [ ]:
# mGdlG <- matrix Grados de libertad global

mNodalCoords = zeros(4, 2)
mNodalCoordsₜ = zeros(4, 2)
FG = zeros(Float64, GDLG)

for el in 1:numElementos
    mGdlE = mGdlG[:, el]
    numGLE = length(mGdlE)
    Uₑ = transpose(reshape(U[mGdlE], 2, 4))
    Fₑ = zeros(Float64, 8, 1)
    
    for nodo in 1:4
        mNodalCoords[nodo, :] = mCoordNodales[mConectG[el, :][nodo], :]
       end

    x₀ = expand((H * mNodalCoords[:, 1])[1])
    y₀ = expand((H * mNodalCoords[:, 2])[1])

    dxdr = expand_derivatives(Dr(x₀))
    dxds = expand_derivatives(Ds(x₀))
    dydr = expand_derivatives(Dr(y₀))
    dyds = expand_derivatives(Ds(y₀))

    J = [dxdr dydr; dxds dyds] # Jacobiano
    J_inv = inv(J) # Inversa del Jacobiano
    J_det = det(J) # Determinante del Jacobiano


    B = J_inv * ∇H


    BL₀ = [B[1, 1] 0 B[1, 2] 0 B[1, 3] 0 B[1, 4] 0
        0 B[2, 1] 0 B[2, 2] 0 B[2, 3] 0 B[2, 4]
        B[2, 1] B[1, 1] B[2, 2] B[1, 2] B[2, 3] B[1, 3] B[2, 4] B[1, 4]]
    


        l₁₁, l₁₂, l₂₁, l₂₂= 0, 0, 0, 0

        for k in 1:4 l₁₁ += B[1, k] * Uₑ[k, 1] end

        for k in 1:4 l₁₂ += B[2, k] * Uₑ[k, 1] end

        for k in 1:4 l₂₁ += B[1, k] * Uₑ[k, 2] end

        for k in 1:4 l₂₂ += B[2, k] * Uₑ[k, 2] end


    BL₁₁ = [l₁₁ * B[1, 1] l₂₁ * B[1, 1] l₁₁ * B[1, 2] l₂₁ * B[1, 2]  l₁₁ * B[1, 3] l₂₁ * B[1, 3] l₁₁ * B[1, 4] l₂₁ * B[1, 4]]
    BL₁₂ = [l₁₂ * B[2, 1] l₂₂ * B[2, 1] l₁₂ * B[2, 2] l₂₂ * B[2, 2]  l₁₂ * B[2, 3] l₂₂ * B[2, 3] l₁₂ * B[2, 4] l₂₂ * B[2, 4]]
    BL₁₃ = [(l₁₁ * B[2, 1] + l₁₂ * B[1, 1]) (l₂₁ * B[2, 1] + l₂₂ * B[1, 1]) (l₁₁ * B[2, 2] + l₁₂ * B[1, 2]) (l₂₁ * B[2, 2] + l₂₂ * B[1, 2]) (l₁₁ *  B[2, 3] + l₁₂ * B[1, 3]) (l₂₁ * B[2, 3] + l₂₂ * B[1, 3]) (l₁₁ * B[2, 4] + l₁₂ * B[1, 4]) (l₂₁ * B[2, 4] + l₂₂ * B[1, 4])]
    BL₁ = vcat(BL₁₁, BL₁₂, BL₁₃)
    BL = BL₀ + BL₁

    # TENSOR GRADIENTE DE DEFORMACIONES

    X₁ = J_inv

    for nodo in 1:4
        mNodalCoordsₜ[nodo, :] = mCoordNodales[mConectG[el, :][nodo], :] + Uₑ[nodo, :]
    end
    
    xₜ = expand((H * mNodalCoordsₜ[:, 1])[1])
    yₜ = expand((H * mNodalCoordsₜ[:, 2])[1])

    dxdrₜ = expand_derivatives(Dr(xₜ))
    dxdsₜ = expand_derivatives(Ds(xₜ))
    dydrₜ = expand_derivatives(Dr(yₜ))
    dydsₜ = expand_derivatives(Ds(yₜ))

    X₂ = [dxdrₜ dydrₜ; dxdsₜ dydsₜ]

    X = X₂ * X₁ # X -> TENSOR GRADIENTE DE DEFORMACIONES

    ϵ = 1 / 2 * (transpose(X) * X - Matrix(I, 2, 2)) # TENSOR DE DEFORMACIONES DE GREEN-LAGRANGE

    ϵv = [ϵ[1,1]; ϵ[2,2]; ϵ[1,2]]

    S = C * ϵv # S -> SEGUNDO TENSOR DE PIOLA KIRCHHOFF
    for i in 1:9
        Sₑₗ = substitute(S, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        BLₑₗ = substitute(BL, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Fₑ += transpose(BLₑₗ) * Sₑₗ
    end

    for i in 1:numGLE # numGLE -> Grados de libertad del elemento
            FG[mGdlE[i]] += Fₑ[i]
    end
end
display(FG)
RG - FG